<a href="https://colab.research.google.com/github/Leriloe/DataAnalysis/blob/main/2_Aggregating_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style='color:blue'> <center>Data Manipulation with Pandas</center> </span>
## **Chapter 2.** Aggregating Data 
#### by **Ivan Alducin**
<p><img src="https://cdn.datafloq.com/cache/blog_pictures/878x531/what-does-clustering-in-data-mining-mean.jpg" width="1250"</p>

## Funciones estadísticas
<p><i>Summary statistics</i> o la estadística de resumen es el conjunto de medidas cuantitativas que nos permiten visualizar el comportamiento de una variable. Por ejemplo, la desviación media, mediana, mínimo, máxima y estándar son estadísticas de resumen. El cálculo de estas nos permite tener una mejor comprensión y entendimiento de nuestro conjunto de datos, incluso si es muy grande.</p>

In [2]:
# Importar pandas y numpy

import pandas as pd
import numpy as np

In [1]:
# Importar el archivo csv sales a Google Colab: se abre botón para cargar archivo.
from google.colab import files
my_up = files.upload()

Saving sales.csv to sales.csv


In [3]:
# Importar el archivo csv sales a un DataFrame
df = pd.read_csv('sales.csv')

In [4]:
# Imprimir las primeras 5 lineas de mi DataFrame
df.head()

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,1,A,1,05/02/2018,24924.50,False,5.727778,0.679451,8.106
1,1,C,1,05/03/2018,21827.90,False,8.055556,0.693452,8.106
2,1,A,1,02/04/2018,57258.43,False,16.816667,0.718284,7.808
3,1,A,1,07/05/2018,17413.94,False,22.527778,0.748928,7.808
4,1,A,1,04/06/2018,17558.09,False,27.050000,0.714586,7.808


In [6]:
# Imprimr la informacion de los campos de mi DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10774 entries, 0 to 10773
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   store                 10774 non-null  int64  
 1   type                  10774 non-null  object 
 2   department            10774 non-null  int64  
 3   date                  10774 non-null  object 
 4   weekly_sales          10774 non-null  float64
 5   is_holiday            10774 non-null  bool   
 6   temperature_c         10774 non-null  float64
 7   fuel_price_usd_per_l  10774 non-null  float64
 8   unemployment          10774 non-null  float64
dtypes: bool(1), float64(4), int64(2), object(2)
memory usage: 684.0+ KB


In [8]:
# Calcular el promedio de las venta semanales
df['weekly_sales'].mean()

23843.950148505668

In [10]:
# Calcular la mediana de las ventas semanales
df['weekly_sales'].median()

12049.064999999999

In [11]:
# Calcular la última fecha de venta: está mal, te da de 2018 pero hay año 2020. 
df['date'].max()

'31/12/2018'

In [14]:
# Hay que cambiar el formato de objeto de la fecha para obtener el mismo valor.
df['date'] = pd.to_datetime(df['date'])
df.info()
df['date'].max()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10774 entries, 0 to 10773
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   store                 10774 non-null  int64         
 1   type                  10774 non-null  object        
 2   department            10774 non-null  int64         
 3   date                  10774 non-null  datetime64[ns]
 4   weekly_sales          10774 non-null  float64       
 5   is_holiday            10774 non-null  bool          
 6   temperature_c         10774 non-null  float64       
 7   fuel_price_usd_per_l  10774 non-null  float64       
 8   unemployment          10774 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 684.0+ KB


Timestamp('2020-12-10 00:00:00')

In [15]:
# Calcular la fecha en la que tuve mi primer venta
df['date'].min()

Timestamp('2018-01-10 00:00:00')

## Multiples funciones
<p>El método <code>.agg()</code> nos permite aplicar nuestras propias funciones personalizadas a un DataFrame, así como también aplicar funciones a más de una columna a la vez, lo que hace que los cálculos sean súper eficientes. Lo anterior se hace usando la siguiente nomenclatura: <code>df['columna'].agg([funcion1, funcion2])</code>
    
En la función personalizada para este ejercicio, <code>IQR</code> nos muestra el rango intercuartil, que es el percentil 75 menos el percentil 25. Es una alternativa a la desviación estándar que resulta útil si sus datos contienen valores atípicos.</p>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Boxplot_vs_PDF.svg/1200px-Boxplot_vs_PDF.svg.png" width="450">

In [16]:
# Funcion para calcular IQR.
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

# Calcular el IQR de la columna 'temperature_c' 
df['temperature_c'].agg(iqr)

16.583333337000003

In [17]:
# Calcula el IQR para 'temperature_c', 'fuel_price_usd_per_l', y 'unemployment'
df[['temperature_c','fuel_price_usd_per_l','unemployment']].agg(iqr)

temperature_c           16.583333
fuel_price_usd_per_l     0.073176
unemployment             0.565000
dtype: float64

In [18]:
# Agrega el promedio al cálculo anterior
# Para llamar una función de panda, se pone entre comillas.
df[['temperature_c','fuel_price_usd_per_l','unemployment']].agg([iqr,'mean'])

,temperature_c,fuel_price_usd_per_l,unemployment
iqr,16.583333,0.073176,0.565000
mean,15.731978,0.749746,8.082009


In [21]:
# Crea un nuevo DataFrame que contenga las ventas de la tienda 1 y el departamento 1
sales_1_1 = df[(df['store']==1) & (df['department']==1)]

In [23]:
# Ordena por fecha
sales_1_1 = sales_1_1.sort_values('date')

# Crea un nuevo campo calculando la venta acumulada semanal
# Te da el acumulado de las weekly_sales
sales_1_1['cum_weekly_sales'] = sales_1_1['weekly_sales'].cumsum()

# Crea un nuevo campo campo calculado con la venta acumulada máxima
sales_1_1['cum_max_sales'] = sales_1_1['weekly_sales'].cummax()

# Vamos a ver los campos que acabas de crear
print(sales_1_1[["date", "weekly_sales", "cum_weekly_sales", "cum_max_sales"]])

         date  weekly_sales  cum_weekly_sales  cum_max_sales
8  2018-01-10      20094.19          20094.19       20094.19
2  2018-02-04      57258.43          77352.62       57258.43
5  2018-02-07      16333.14          93685.76       57258.43
7  2018-03-09      16241.78         109927.54       57258.43
10 2018-03-12      22517.56         132445.10       57258.43
4  2018-04-06      17558.09         150003.19       57258.43
0  2018-05-02      24924.50         174927.69       57258.43
1  2018-05-03      21827.90         196755.59       57258.43
9  2018-05-11      34238.88         230994.47       57258.43
6  2018-06-08      17508.41         248502.88       57258.43
3  2018-07-05      17413.94         265916.82       57258.43
11 2019-07-01      15984.24         281901.06       57258.43


## Eliminando duplicados de un DataFrame
<p>Eliminar duplicados es una habilidad esencial para obtener análisis precisos porque, a menudo, no deseamos tener valores repetidos dentro de nuestros cálculos.</p>

In [24]:
# Elimina los duplicados de las combinaciones store/type
store_types = df.drop_duplicates(subset=['store','type'])

# Resultado
store_types.head(15)

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,1,A,1,2018-05-02,24924.50,False,5.727778,0.679451,8.106
1,1,C,1,2018-05-03,21827.90,False,8.055556,0.693452,8.106
901,2,A,1,2018-05-02,35034.06,False,4.550000,0.679451,8.324
914,2,C,2,2018-05-03,68428.64,False,8.427778,0.693452,8.324
1798,4,A,1,2018-05-02,38724.42,False,6.533333,0.686319,8.623
2681,4,C,98,2018-02-07,13089.72,False,23.144444,0.704811,7.372
2699,6,A,1,2018-05-02,25619.00,False,4.683333,0.679451,7.259
2886,6,C,17,2018-03-09,16640.20,False,27.861111,0.680772,6.973
3593,10,B,1,2018-05-02,40212.84,False,12.411111,0.782478,9.765
4495,13,A,1,2018-05-02,46761.90,False,-0.261111,0.704283,8.316


In [25]:
# Elimina los duplicados de las combinaciones store/department y ordena por venta semanal
store_depts =  df.drop_duplicates(subset=['store','department']).sort_values('weekly_sales')

# Resultado
store_depts.head()

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
6806,19,A,47,2018-04-06,-70.00,False,20.488889,0.794102,8.185
3215,6,A,47,2018-05-02,-59.00,False,4.683333,0.679451,7.259
7900,20,A,77,2019-04-15,-29.97,False,12.777778,1.009402,7.287
9691,31,A,78,2018-05-02,-12.00,False,3.916667,0.679451,8.324
10103,39,A,19,2018-05-14,-11.00,False,25.272222,0.753947,8.464


In [26]:
# Filtra las filas en donde las ventas hayan sido en días festivos ('is_holiday') y borra los duplicados de las fechas
holiday_dates = df[df['is_holiday']==True].drop_duplicates('date')

# Resultado
holiday_dates.head()

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
498,1,A,45,2018-10-09,11.47,True,25.938889,0.677602,7.787
691,1,A,77,2019-11-25,1431.00,True,15.633333,0.854861,7.866
2315,4,A,47,2018-12-02,498.00,True,-1.755556,0.679715,8.623
6735,19,A,39,2020-07-09,13.41,True,22.333333,1.076766,8.193
6810,19,A,47,2018-12-31,-449.00,True,-1.861111,0.881278,8.067


## Conteo y proporción
<p>Contar es una excelente manera de obtener una descripción general de nuestro conjunto datos y poder analizar sobre todo nuestras categorías top o en su defecto las que menor proporción tienen con respecto una variable</p>

In [27]:
# Contar el número de cada tipo de tiedas
df['type'].value_counts()

A    9766
B     902
C     106
Name: type, dtype: int64

In [30]:
# Dar la proporcion del cálculo anterior (porcentaje)
round(df['type'].value_counts(normalize=True)*100,2)


A    90.64
B     8.37
C     0.98
Name: type, dtype: float64

In [32]:
# Contar por tienda y ordenar el resultado (potcentaje)
df['store'].value_counts(normalize=True, sort=True)*100


13    8.474104
20    8.446260
19    8.409133
10    8.372007
4     8.362725
1     8.362725
27    8.353443
2     8.325599
6     8.297754
31    8.260627
14    8.214219
39    8.121403
Name: store, dtype: float64

In [33]:
# Dar la proporción ordenada del cálculo anterior
df['store'].value_counts(normalize=True, sort=True)


13    0.084741
20    0.084463
19    0.084091
10    0.083720
4     0.083627
1     0.083627
27    0.083534
2     0.083256
6     0.082978
31    0.082606
14    0.082142
39    0.081214
Name: store, dtype: float64

## Porcentaje de ventas
<p>Walmart distingue tres tipos de tiendas: "supercenters", "discount stores" y "neighborhood markets", codificados en este conjunto de datos como tipo "A", "B" y "C". En este ejercicio, vamos a calcular las ventas totales realizadas en cada tipo de tienda, sin utilizar <code>.groupby()</code>.</p>

In [35]:
# Calcular el total de ventas semanales
sales_all = df['weekly_sales'].sum()

# Calcular las ventas semanales para supercenters
sales_A = df[df["type"] == "A"]["weekly_sales"].sum()

# Calcular las ventas semanales para discount stores (minoristas)
sales_B = df[df["type"] == "B"]["weekly_sales"].sum()


# Calcular las ventas semanales para neighborhood markets
sales_C = df[df["type"] == "C"]["weekly_sales"].sum()


# Calcular la proporción de venta por tipo de tienda
prop = [sales_A, sales_B, sales_C]/sales_all*100
prop.round(2)


array([89.97,  9.02,  1.  ])

## Group by
<p> Podemos ahorrarnos mucho código usando el método <code>.groupby</code> como veremos a continuación</p>

In [36]:
# Calcular las ventas por tipo de tienda usando .groupby()
sales_by_type = df.groupby("type")["weekly_sales"].sum()

# Calcular la proporción
sales_propn_by_type = sales_by_type/sales_all

# Resultado
sales_propn_by_type

type
A    0.899738
B    0.090225
C    0.010036
Name: weekly_sales, dtype: float64

In [37]:
# Para cada tipo de tienda calcular la venta nínima, máxima, el promedio y la media (Usar las funciones NumPy)
r = df.groupby('type')['weekly_sales'].agg([np.min,np.max,np.mean])

# Resultado: negativos = devoluciones
r

,amin,amax,mean
type,,,
A,-1098.00,293966.05,23667.624573
B,-798.00,232558.51,25696.678370
C,0.01,104161.40,24323.522925


In [39]:
# Para cada tipo de tienda, agrega los campos 'unemployment' y 'fuel_price_usd_per_l' y calcula el min, max, promedio y mediana
#Vaciar variable
r = None
r = df.groupby('type')['weekly_sales','unemployment','fuel_price_usd_per_l'].agg([np.min,np.max,np.mean])

# Resultado
r

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


weekly_sales             ... fuel_price_usd_per_l          
             amin       amax  ...                 amax      mean
type                          ...                               
A        -1098.00  293966.05  ...             1.107410  0.744943
B         -798.00  232558.51  ...             1.107674  0.805858
C            0.01  104161.40  ...             0.921433  0.714793

[3 rows x 9 columns]

## Pivot Tables
<p> Las tablas dinámicas son la forma estándar de agregar datos en hojas de cálculo. En pandas, las tablas dinámicas son esencialmente otra forma de realizar cálculos agrupados. Es decir, el método <code>.pivot_table()</code> es justamente una alternativa a <code>.groupby()</code>.</p>

In [40]:
# Calcular el promedio de ventas semanales por tipo de tienda con el metodo .pivot_table() 
pivot = df.pivot_table(values='weekly_sales', index='type',aggfunc=np.mean)

# Resultado
pivot

,weekly_sales
type,
A,23667.624573
B,25696.678370
C,24323.522925


In [41]:
# Calcular el promedio y la mediana de ventas semanales por tipo de tienda 
pivot = df.pivot_table(values='weekly_sales', index='type',aggfunc=[np.mean,np.median])

# Resultado
pivot

,mean,median
,weekly_sales,weekly_sales
type,,
A,23667.624573,11910.165
B,25696.678370,13336.080
C,24323.522925,16643.205


In [44]:
# Calcular el promedio de ventas semanales por tipo de tienda y departamento
pivot = df.pivot_table(values='weekly_sales', index='department',aggfunc=np.mean, columns='type')

# Resultado
pivot

type,A,B,C
department,,,
1,31145.215923,44050.626667,19034.840000
2,67736.748231,112958.526667,58721.845000
3,17124.044538,30580.655000,19497.300000
4,44265.151679,51219.654167,46937.810000
5,34942.639618,63236.875000,18887.710000
...,...,...,...
95,124084.721374,77082.102500,104161.400000
96,21184.264918,9528.538333,26941.770000
97,28503.679612,5828.873333,27077.523333


In [45]:
# Calcular el promedio de ventas semanales por tipo de tienda y departamento, reemplazando los valores nulos con 0
pivot = df.pivot_table(values='weekly_sales', index='department',aggfunc=np.mean, columns='type', fill_value=0)

# Resultado
pivot

type,A,B,C
department,,,
1,31145.215923,44050.626667,19034.840000
2,67736.748231,112958.526667,58721.845000
3,17124.044538,30580.655000,19497.300000
4,44265.151679,51219.654167,46937.810000
5,34942.639618,63236.875000,18887.710000
...,...,...,...
95,124084.721374,77082.102500,104161.400000
96,21184.264918,9528.538333,26941.770000
97,28503.679612,5828.873333,27077.523333


In [ ]:
# Calcular el promedio de ventas semanales por tipo de tienda y departamento, reemplazando los valores nulos con 0 y sumando las filas y columnas
______

# Resultado
______